In [1]:
from pathlib import Path
import numpy as np
import logging

import pyvista as pv
pv.set_jupyter_backend("pythreejs")

from project_heart.enums import *
from project_heart.lv import LV
from project_heart.examples import get_lv_ideal, get_lv_typeA

from project_heart.utils.tensor_utils import convert_to_cylindrical_coordinates

directory = Path("C:/Users/igorp/OneDrive - University of South Florida/Igor/Documents/Paper 1/stress_strain_plots/ref_files")
# filename = "57.32_40.94_10.08.2022_LVRNN_IDEAL_V4"
filename = "62.71_68.65_10.08.2022_LVRNN_TYPEA_V4"

lv = get_lv_typeA(str(directory/filename) + ".xplt", statesfile=None)

INFO:LV_RegionIdentifier.identify_regions_typeA:Identifying regions from 'type A' geometry.
INFO:LV_RegionIdentifier.identify_endo_epi_base_borders_from_nodesets:len(endo_base): 144
INFO:LV_RegionIdentifier.identify_endo_epi_base_borders_from_nodesets:len(epi_base): 144


In [2]:
cy_stress = lv.convert_to_cylindrical_coordinates(lv.STATES.STRESS)
cy_strain = lv.convert_to_cylindrical_coordinates(lv.STATES.LAGRANGE_STRAIN)

In [3]:
idx = np.arange(1,cy_stress.shape[1]+1).reshape(-1,1)

In [4]:
np.savetxt(
    str(directory/filename) + "_cylindrical_stress_xplt_format_2D_angle_with_orientation.csv", 
    np.hstack((idx, np.hstack(cy_stress))), 
    delimiter=",", fmt="%.3f"
    )

In [5]:
np.savetxt(
    str(directory/filename) + "_cylindrical_strain_xplt_format_2D_angle_with_orientation.csv", 
    np.hstack((idx, np.hstack(cy_strain))), 
    delimiter=",", fmt="%.3f"
    )

In [ ]:
from project_heart.utils.vector_utils import angle_between

# set references
x_axis = np.asarray([1,0,0])
z_axis = np.asarray([0,0,1])
ref_pt = np.asarray([0,0,0])
    
displacements = lv.states.get(lv.STATES.DISP)
all_thetas = []
for ts, disp in enumerate(displacements):
    tmp_mesh = lv.mesh.copy()
    tmp_mesh.points += disp
    centers = np.asarray(tmp_mesh.cell_centers().points)
   
    # create vector from element centers
    u = centers - ref_pt

    # comput angle with respect to x_axis
    thetas = angle_between(u[:, :2], x_axis[:2], check_orientation=True, zaxis=z_axis)
    all_thetas.append(thetas)

In [ ]:
all_thetas = np.asarray(all_thetas)
all_thetas.shape

In [ ]:
all_thetas[0]

In [ ]:
np.savetxt(
    str(directory/filename) + "_thetas_2D_with_angle_orientation.csv", 
    np.hstack((idx, np.hstack(all_thetas.reshape((163, -1, 1))))), 
    delimiter=",", fmt="%.3f"
    )